# RotD50 RotD100 for <font color='red'>multiple loop</font> 

### - The details are in the Boore's paper (2010BSSA)


#### - Import the modules used in this program

In [ ]:
from obspy.core import read
from obspy.core.trace import Trace
from obspy.signal.rotate import rotate_ne_rt
import matplotlib.pyplot as plt
import numpy as np

### - Define the cut time window function

In [ ]:
 def cut_window(array, dt, beg, end):
        temp = Trace(data=array)
        temp.stats.delta = dt
        t0 = temp.stats.starttime
        temp.trim(t0 + beg, t0 + end)
        out = temp.data
        return out

###  <font color='red'>- Calculate RotD50 and RotD100 for each station</font> 

In [ ]:
def RotD_cal(file_accEW, file_accNS, file_velEW, file_velNS):
    
    '''
    Input:
        file_accEW: EW-comp acceleration sac file
        file_accNS: NS-comp acceleration sac file
        file_velEW: EW-comp velocity sac file
        file_velNS: NS-comp velocity sac file
        
    Output:
        RotD50_PGA
        RotD100_PGA
        RotD50_PGV
        RotD100_PGV
    '''

    data_accEW = read(file_accEW)
    data_accNS = read(file_accNS)
    data_velEW = read(file_velEW)
    data_velNS = read(file_velNS)

    # Create time window
    dt = data_accEW[0].stats.sac.delta
    Num = data_accEW[0].stats.npts
    x = np.linspace(0,Num*dt,Num)

    # Retrieve start and end time
    beg = data_accEW[0].stats.sac.a
    end = data_accEW[0].stats.sac.f
    b = data_accEW[0].stats.sac.b
    beg = beg - b
    end = end - b
    
    # Cut time window
    t = cut_window(x, dt, beg, end)
    accEW = cut_window(data_accEW[0].data, dt, beg, end)
    accNS = cut_window(data_accNS[0].data, dt, beg, end)
    velEW = cut_window(data_velEW[0].data, dt, beg, end)
    velNS = cut_window(data_velNS[0].data, dt, beg, end)

    
    # Rotate for each 1-deg increment
    # Rotation-angle periodicity of 180 deg
    
    #---Notice!!---#
    # Basically it's good to use numpy module, if you calculate with array.
    # But you have to prepare the empty array, when you use it.

    #Prepare the empty array
    max_acc = np.array([0.0]*180)
    max_vel = np.array([0.0]*180)
    
    # Calculate Peak Amplitude over the whole rotation angles
    # For PGA
    for i in range(0, 180):
        (rad, trans) = rotate_ne_rt(accNS, accEW, i)
        ab_rad = abs(rad) 
        max_acc[i] = np.max(ab_rad)
    
    # For PGV
    for i in range(0, 180):
        (rad, trans) = rotate_ne_rt(velNS, velEW, i)
        ab_rad = abs(rad) 
        max_vel[i] = np.max(ab_rad)
    
    ang = np.linspace(1,180,180)

    # Pick up the maximum (RotD100) and median (RotD50) values
    # Multiply 100 from m/sec^2 to gal for PGA and m/sec to cm/sec
    RotD50_PGA = np.median(max_acc) * 100.0
    RotD100_PGA = np.max(max_acc) * 100.0
    RotD50_PGV = np.median(max_vel) * 100.0
    RotD100_PGV = np.max(max_vel) * 100.0

    return RotD50_PGA, RotD100_PGA, RotD50_PGV, RotD100_PGV


## Main loop

In [ ]:
import math
#### Read the files from "list.txt"
#### For each station:
#### accEW, accNS, velEW, velNS in each column
infile = input()

# How many stations do you have in this event?
num_st = sum(1 for line in open(infile))

# Create the empty array
Station = np.array(["Station"]*num_st)
Dist = np.array([0.0]*num_st)
HypoDist = np.array([0.0]*num_st)
Az = np.array([0.0]*num_st)
RotD50_PGA = np.array([0.0]*num_st)
RotD100_PGA = np.array([0.0]*num_st)
RotD50_PGV = np.array([0.0]*num_st)
RotD100_PGV = np.array([0.0]*num_st)
Stla = np.array([0.0]*num_st)
Stlo = np.array([0.0]*num_st)


# Read the event header from SAC data
f = open(infile)
# Read for 1st column
f1 = f.readline()
f11, f12, f13, f14 = f1.split()
sac = read(f11)

hdr_event = str(sac[0].stats.starttime)
hdr_event = hdr_event[0:19]
hdr_mag = sac[0].stats.sac.mag
hdr_depth = sac[0].stats.sac.evdp
hdr_evla = sac[0].stats.sac.evla
hdr_evlo = sac[0].stats.sac.evlo

EventID = np.array([hdr_event]*num_st)
Mag = np.array([hdr_mag]*num_st)
Depth = np.array([hdr_depth]*num_st)
Evla = np.array([hdr_evla]*num_st)
Evlo = np.array([hdr_evlo]*num_st)


# Main loop
import fileinput
i = -1
for line in fileinput.input(infile):
    file_accEW, file_accNS, file_velEW, file_velNS = line.split()
    print (file_accEW, file_accNS, file_velEW, file_velNS)

    # Read SAC Data
    sac = read(file_accEW)
    # Check the begin and end time
    # If SAC file doesn't include these header vaules, skip this station
    try:
           beg = sac[0].stats.sac.a
    except:
           continue
    try:
           end = sac[0].stats.sac.f
    except:
           continue

    # Count the station
    i = i + 1
    
    # Get the station header information
    Station[i] = sac[0].stats.station
    Dist[i] = sac[0].stats.sac.dist
    Az[i] = sac[0].stats.sac.az
    Stla[i] = sac[0].stats.sac.stla
    Stlo[i] = sac[0].stats.sac.stlo

    
    # Calculate the hypocentral distance
    dep = sac[0].stats.sac.evdp
    HypoDist[i] = math.sqrt(dep**2 + Dist[i]**2)
    
    
    # Get the PGA PGV data 
    RotD50_PGA[i], RotD100_PGA[i], RotD50_PGV[i], RotD100_PGV[i] = RotD_cal(file_accEW, file_accNS, file_velEW, file_velNS)

    
# Create data_array for output  
array_Name = np.c_[Station, EventID]
array_GM = np.c_[RotD50_PGA, RotD100_PGA, RotD50_PGV, RotD100_PGV]
array_Info = np.c_[Dist, HypoDist, Az, Mag, Depth, Stla, Stlo, Evla, Evlo]

# Shrink the array size due to the skip of some stations
array_Name = array_Name[:i+1,:]
array_GM = array_GM[:i+1,:]
array_Info = array_Info[:i+1,:]

print(array_Name)

# Convert to pandas data frame
import pandas as pd
Table_Name = pd.DataFrame(array_Name)
Table_Name.columns = ["Station", "EventID"]
print(Table_Name)
Table_GM = pd.DataFrame(array_GM)
Table_GM.columns = ["RotD50_PGA", "RotD100_PGA", "RotD50_PGV", "RotD100_PGV"]
Table_Info = pd.DataFrame(array_Info)
Table_Info.columns = ["Dist", "HypoDist", "Az", "Mag", "Depth", "Stla", "Stlo", "Evla", "Evlo"]
# Change the significant figures
Table_Info = Table_Info.round(3)

# Combine Tables
Table = pd.concat([Table_Name, Table_GM, Table_Info], axis=1)

# Output for csv file
EV = EventID[0]
YYYYMMDD = EV[0:9]
HH = EV[11:12]
MM = EV[14:15]
SS = EV[17:18]
#Name_Table = "".join(["PGA_PGV_",EventID[0],".csv"])
Name_Table = "".join(["PGA_PGV_",YYYY-MM-DD,"_",HH,"-",MM,"-",SS,".csv"])
Table.to_csv(Name_Table)
    